<a href="https://colab.research.google.com/github/mmridull/mlpolicicyhub/blob/main/Mlpolicyhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Train an Intent Classification Model** feature extraction and text classification using skitlearn

In [ ]:
!pip install scikit-learn numpy pandas
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

with open("/content/intents.json") as f:
    data = json.load(f)

tags = []
patterns = []
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tags.append(intent["tag"])
        patterns.append(pattern)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(patterns)
y = np.array(tags)

model = LinearSVC()
model.fit(X, y)

LinearSVC()

**Saveing the Model and Vectorizer**

In [ ]:
import joblib
joblib.dump(model, "intent_classifier.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

['tfidf_vectorizer.pkl']

** Integrateing ML Model into Chatbot**

In [ ]:
!pip install gradio
import openai
import gradio as gr
import joblib

model = joblib.load("intent_classifier.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

company_policies = {
    "work_from_home": "Employees can work from home 3 days a week with manager approval.",
    "leave_policy": "20 paid leave days per year. Unused days carry over (max 10 days).",
    "dress_code": "Business casual. Formal attire for client meetings.",
}

def classify_intent(user_input):
    user_input_vec = vectorizer.transform([user_input])
    intent = model.predict(user_input_vec)[0]
    return intent

def chatbot_response(user_input):
    intent = classify_intent(user_input)
    if intent in company_policies:
        return company_policies[intent]
    else:
        response = openai.Completion.create(
            engine="text-davinci-003",
            prompt=f"Answer this question about company policies: {user_input}",
            max_tokens=100
        )
        return response.choices[0].text.strip()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


**Interface**

In [ ]:
def chat_interface(user_input, history):
    history = history or []
    response = chatbot_response(user_input)
    history.append((user_input, response))
    return history, history

iface = gr.Interface(
    fn=chat_interface,
    inputs=["text", "state"],
    outputs=["chatbot", "state"],
    title="Company Policy Chatbot (ML-Powered)",

)

iface.launch()

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://33112ddfd0989177df.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
